# Voronoi diagram 0.0

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import sys
from matplotlib import collections as mc
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.cm import jet
from collections import defaultdict

In [2]:
class Ordering(object):

    def go_to_i(alist, elem, comparison):
        i = 0
        while i < len(alist) and comparison(elem, alist[i]):
            i += 1
        return i
    
    def is_coord_y_greater(elem1, elem2):
        return elem1.coord.y>elem2.coord.y

In [3]:
class Circle():
    
    def center(a, b, c):
        d = (a.x - c.x) * (b.y - c.y) - (b.x - c.x) * (a.y - c.y)
        if d==0:
            return Point(np.inf, np.inf)
        xc = (((a.x - c.x) * (a.x + c.x) + (a.y - c.y) * (a.y + c.y)) / 2 *
              (b.y - c.y) - ((b.x - c.x) * (b.x + c.x) + (b.y - c.y) * (b.y + c.y)) / 2 *
              (a.y - c.y))/d
        yc = (((b.x - c.x) * (b.x + c.x) + (b.y - c.y) * (b.y + c.y)) / 2 *
              (a.x - c.x) - ((a.x - c.x) * (a.x + c.x) + (a.y - c.y) * (a.y + c.y)) / 2 *
              (b.x - c.x))/d
        return Point(xc, yc)

    def bottom(a, b, c):
        cc = self.center(a, b, c)
        return Point(cc.x, cc.y-cc.dist_to_point(a))

    def top(a, b, c):
        cc = self.center(a, b, c)
        return Point(cc.x, cc.y+cc.dist_to_point(a))

In [4]:
class Parabol():
    def cross_x(f1, f2, q):
        if f1.y!=f2.y:
            s1=(f1.y*f2.x-f1.x*f2.y+f1.x*q-f2.x*q+np.sqrt((f1.x*f1.x+f1.y*f1.y-2*f1.x*f2.x+f2.x*f2.x-2*f1.y*f2.y+f2.y*f2.y)*(f1.y*f2.y-f1.y*q-f2.y*q+q*q)))/(f1.y-f2.y)
            s2=(f1.y*f2.x-f1.x*f2.y+f1.x*q-f2.x*q-np.sqrt((f1.x*f1.x+f1.y*f1.y-2*f1.x*f2.x+f2.x*f2.x-2*f1.y*f2.y+f2.y*f2.y)*(f1.y*f2.y-f1.y*q-f2.y*q+q*q)))/(f1.y-f2.y)
            return [s1, s2] if s1<s2 else [s2, s1]
        else:
            return [(f1.x+f2.x)/2, (f1.x+f2.x)/2]

    def y(f, q):
        def y(x):
            if q!=f.y:
                return np.power((x - f.x),2)/(2*(abs(q-f.y)))+(q-f.y)/2
            else:
                return np.inf
        return y

In [5]:
class Point(object):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __str__(self):
        return "(%.2f,%.2f)" % (self.x,self.y)

    def __repr__(self):
        return str(self)

    def __eq__(self, other):
        return isinstance(other, Point) and self.x==other.x and self.y==other.y  
    
    def __hash__(self):
        return hash(str(self))
        
    def is_x_greater(self, p):
        return self.x > p.x
    
    def is_y_greater(self, p):
        return self.y > p.y
        
    def delta_x(self, p):
        return self.x-p.x
    
    def delta_y(self, p):
        return self.y-p.y
    
    def dist_to_point(self, p):
        return np.sqrt(np.power(self.x - p.x, 2) + np.power(self.y - p.y, 2))
    
    def dist_to_par(self, focus): #se la distanza punto parabola è infinita (direttrice parabola passante per punto e fuoco)
        return np.power(self.dist_to_point(focus), 2)/(2*abs(self.delta_y(focus))) if self.delta_y(focus)!=0 else np.inf
        
    def is_in(self, ps):
        for p in ps:
            if self==p:
                return True
        return False

In [6]:
print("1st intersection:", Parabol.cross_x(Point(0,0), Point(4,2), 4)[0],". Second intersection: ",Parabol.cross_x(Point(0,0), Point(4,2), 4)[1])

1st intersection: 1.67544467966 . Second intersection:  14.3245553203


In [7]:
f=Point(0,0)
q=2
x=2
print("Parabola with y =", q, "and focus in",str(f),"evaluated in x =",x,"gives f(x) =", Parabol.y(f,q)(x))

Parabola with y = 2 and focus in (0.00,0.00) evaluated in x = 2 gives f(x) = 2.0


In [8]:
class Segment(object):
    
    def __init__(self,  pl=None, pr=None, start=None, end=None):
        self.start = start #punto iniziale del segmento
        self.end = end #punto finale del segmento
        self.pl = pl #uno dei due punti di cui l'oggetto è un segmento dell'asse
        self.pr = pr #uno dei due punti di cui l'oggetto è un segmento dell'asse
        
    def __str__(self):
        start = "None" if self.start==None else self.start
        end = "None" if self.end==None else self.end
        return "(%s -> %s, left focus %s, right focus %s)" % (start, end, self.pl, self.pr)
    
    def halfplane(self):
        #QUI SFRUTTA L'ORDINAMENTO IN Y DEGLI EVENTI: i semisegmenti vengono specchiati rispetto a x se cambia il verso di sweep
        #per sweep verso x positive. +: semipiano superiore e x+; -: semipiano inferiore e x-
        #per sweep verso x negative. +: semipiano inferiore e x+; -: semipiano superiore e x- (possibile invertire la condizione)
        #L'assegnazione di x+ e x- è stata scelta arbitrariamente
        if self.pl.x<self.pr.x: #semipiano superiore
            return 2
        if self.pl.x>self.pr.x: #semipiano inferiore
            return -2
        if self.pl.x==self.pr.x and self.pl.y>self.pr.y: #asse x+
            return 1
        return -1 #asse x-
    
    def m(self): #y=mx+q ; m=-(pl.x-pr.x)/(pl.y-pr.y) ; q=(pl.y+pr.y)/2-m*(pl.x+pr.x)/2
        return np.inf if self.pl.y==self.pr.y else -(self.pl.x-self.pr.x)/(self.pl.y-self.pr.y)
    
    def direction(self):
        (m, hp)=(self.m(), self.halfplane())
        return Point(1,m) if (hp*m>0 or (m==0 and hp>0)) else Point(-1,-m) 
    
    def theta(self, rad=False):
        theta = np.arctan2(self.direction().y, self.direction().x)
        return np.degrees(theta) if rad==False else theta
        
    def y(self, x):
        return self.m()*x+self.start.y-self.m()*self.start.x
    
    def get_intersection(s1, s2=None):
        (s1_m, s2_m)=(s1.m(), s2.m())
        (s1_hp, s2_hp)=(s1.halfplane(), s2.halfplane())
        #print(str(s1_m), str(s2_m), str(s1_hp), str(s2_hp))
        if (s1_hp*s1_m>0 or (s1_m==0 and s1_hp>0)): #semiretta semipiano dx 
            s1_vec=Point(1,s1_m) 
        else: #semiretta nel semipiano sx
            s1_vec=Point(-1,-s1_m)
        if (s2_hp*s2_m>0 or (s2_m==0 and s2_hp>0)): #semiretta semipiano dx 
            s2_vec=Point(1,s2_m) 
        else: #semiretta nel semipiano sx
            s2_vec=Point(-1,-s2_m)
        dx = s2.start.x - s1.start.x
        dy = s2.start.y - s1.start.y
        if s2_m==np.inf: #segmento 2 verticale
            #print("Vertical:", s2_hp*(s1_m*dx-dy), s1_vec.x*dx)
            #1e10 sarebbe 0, ma è inserito uno scarto per approssimazioni calcoli con i float
            #Tale approssimazione rende i confronti più robusti, ma nel caso in cui le rette
            #partono dallo stesso start fa risultare che le stesse non si intersecano
            if s2_hp*(s1_m*dx-dy)>=-1.0e-10 and s1_vec.x*dx>=-1.0e-10: 
                return Point(s2.start.x, s1.start.y+s1_m*(dx))
            return None
        if s1_m==np.inf: #segmento 1 verticale; stesse condizioni ma con s1->s2, dx->-dx, dy->-dy
            #print("Vertical ",s1_hp*(s2_m*dx-dy), s2_vec.x*dx)
            if s1_hp*(s2_m*dx-dy)<=1.0e-10 and s2_vec.x*dx<=1.0e-10:
                return Point(s1.start.x, s2.start.y-s2_m*(dx))
            return None
        det = s2_vec.x * s1_vec.y - s2_vec.y * s1_vec.x #ritorna False se semirette //, anche se collineari
        if det==0:
            return None
        u = (dy * s2_vec.x - dx * s2_vec.y) / det
        v = (dy * s1_vec.x - dx * s1_vec.y) / det
        #print("Not vertical ", str(u), str(v))
        if (u>=-1e-10 and v>=1e-10) or (u>=1e-10 and v>=-1e-10):
            return Point(s1.start.x+u*s1_vec.x, s1.start.y+u*s1_vec.y)
        return None

In [9]:
s1=Segment(start=Point(0,0), pl=Point(1,0), pr=Point(0,1))
s2=Segment(start=Point(0,0), pl=Point(1,0), pr=Point(-1,0))
cross_point=s1.get_intersection(s2)
if cross_point!=None:
    print("Cross point:", str(cross_point))

Cross point: (0.00,0.00)


In [10]:
class Beachline(object):
    
    def __init__(self, item=None):
        self.item = item
        self.parent = None
        self.l = None
        self.r = None
        self.c_event=None
        
    def str_node(self, directions=[]):
        cur_node=self
        for d in directions:
            cur_node=cur_node.l if d=="l" else cur_node.r
        if isinstance(cur_node.item, Segment):
            return "Edge. "+str(cur_node.item)
        else:
            return "Arc. "+str(cur_node.item)+". Circle event: None" if cur_node.c_event==None else "Arc. "+str(cur_node.item)+". Event: "+str(cur_node.c_event)
    
    def str_tree(self, path="Path: root"):
        if self.item==None: #un po' inefficente fargli compiere l'operazione ad ogni iterazione
            return "Empty beachline.\n"
        node_string=path+". "+self.str_node()+"\n"
        if not self.is_leaf():
            node_string+=self.l.str_tree(path+"->l")
            node_string+=self.r.str_tree(path+"->r")
        return node_string
    
    def get_leaves(self):
        cur_node=self
        leaves=[]
        while not cur_node.is_leaf():
            cur_node=cur_node.l
        while cur_node!=None:
            leaves.append(cur_node)
            cur_node=cur_node.get_rleaf_and_rparent()[0]
        return leaves
              
    def is_leaf(self):
        return self.l==None
    
    def is_lchild(self):
        return not self.is_root() and self.parent.l==self
    
    def is_rchild(self):
        return not self.is_root() and self.parent.r==self
                
    def is_root(self):
        return self.parent==None
        
    def add_rchild(self, item, event=None):
        child = Beachline(item)
        child.parent = self
        self.r = child
        
    def add_lchild(self, item, event=None):
        child = Beachline(item)
        child.parent = self
        self.l = child
        
    def get_nearest_arc_node_by_x(self, new_site):
        if self.item==None: #beachline vuota. In tal caso non esiste un sito più vicino
            return None
        else:
            cur_node=self
        while isinstance(cur_node.item, Segment): #essendo al primo passaggio, siteA.x!=siteB.x, per ogni sito
            cur_node=cur_node.l if new_site.x<(cur_node.item.pl.x+cur_node.item.pr.x)/2 else cur_node.r
        return cur_node
    
    def get_arc_node_on_site(self, new_site):
        if self.item==None: #beachline vuota. In tal caso non esiste un arco sopra il sito aggiunto, Non dovrebbe verificarsi mai
            return None
        else:
            cur_node=self
        cur_node = self
        while isinstance(cur_node.item, Segment): #se il nuovo sito è equidistante da pl e pr è automaticamente associato al figlio destro (non so se corretto)
            #print("Intersection between",str(cur_node.item.pl),"and",str(cur_node.item.pr),"when q=",str(new_site.y),": ", str(Parabol.cross_x(cur_node.item.pl, cur_node.item.pr, new_site.y)))
            sol=0 if cur_node.item.pl.y<cur_node.item.pr.y else 1
            cur_node=cur_node.l if new_site.x<(Parabol.cross_x(cur_node.item.pl, cur_node.item.pr, new_site.y)[sol]) else cur_node.r
        return cur_node
    
    def get_lleaf_and_lparent(self, leaf=None):
        cur_node=self if leaf==None else leaf #se non passo un argomento al metodo utilizza come foglia il nodo chiamante
        while cur_node.is_lchild():
            cur_node=cur_node.parent
        if cur_node.is_root():
            return None, None
        ledge=cur_node.parent
        cur_node=cur_node.parent.l
        while not cur_node.is_leaf():
            cur_node=cur_node.r
        return cur_node, ledge
    
    def get_rleaf_and_rparent(self, leaf=None):
        cur_node=self if leaf==None else leaf #se non passo un argomento al metodo utilizza come foglia il nodo chiamante
        while cur_node.is_rchild():
            cur_node=cur_node.parent
        if cur_node.is_root():
            return None, None
        redge=cur_node.parent
        cur_node=cur_node.parent.r
        while not cur_node.is_leaf():
            cur_node=cur_node.l
        return cur_node, redge  
    
    #Deve essere chiamato per aggiungere gli archi iniziali, cioè quagli archi associati ai primi eventi sito incontrati alla stessa y
    #Non necessario se l'algoritmo è chiuso aggiungendo 4 punti a rombo sufficientemente distanti dal range di visualizzazione (1 solo primo 
    #evento sito alla y iniziale)
    #Ha all'incirca l'effetto cumulato di arr_arc seguito da rm_arc, senza la parte di gestione degli eventi cerchio. Introduce una 
    #lieve asimmetria rispetto alla restante interfaccia in quanto non si fa passare il nodo del segmento modificato (come fa rm_arc),
    #siccome tale dato non serve già al metodo più esterno che gestisce gli eventi sito
    def add_start_arc(self, new_arc_site, nearest_arc_node=None): 
        if nearest_arc_node==None: #mi è stato passato il primo sito in assoluto
            self.item=new_arc_site
            return
        nearest_arc_site=nearest_arc_node.item
        if new_arc_site.x<nearest_arc_site.x: #nuovo sito inserito come figlio sx; sito più vicino in x come figlio dx
            #aggiornamento nodo segmento che tracciava l'asse tra i due siti fra cui è stato inserito il nuovo sito
            top_updated_edge_node=nearest_arc_node.get_lleaf_and_lparent()[1]
            if top_updated_edge_node!=None: #se il sito inserito non è quello più a sx
                top_updated_edge_node.item.pr=new_arc_site
                top_updated_edge_node.item.start.x=(new_arc_site.x+top_updated_edge_node.item.pl.x)/2
            #costruzione del nuovo ramo
            new_edge=Segment(pl=new_arc_site, pr=nearest_arc_site, start=Point((new_arc_site.x+nearest_arc_site.x)/2, -np.inf))
            nearest_arc_node.item=new_edge
            nearest_arc_node.add_lchild(new_arc_site)
            nearest_arc_node.add_rchild(nearest_arc_site)
        else: #nuovo sito inserito come figlio dx; sito più vicino in x come figlio sx
            top_updated_edge_node=nearest_arc_node.get_rleaf_and_rparent()[1]
            if top_updated_edge_node!=None:
                top_updated_edge_node.item.pl=new_arc_site
                top_updated_edge_node.item.start.x=(new_arc_site.x+top_updated_edge_node.item.pr.x)/2
            new_edge=Segment(pl=nearest_arc_site, pr=new_arc_site, start=Point((new_arc_site.x+nearest_arc_site.x)/2, -np.inf))
            nearest_arc_node.item=new_edge
            nearest_arc_node.add_rchild(new_arc_site)
            nearest_arc_node.add_lchild(nearest_arc_site)
            
    #modifica la beach in modo che, dove prima c'era il sito arco intersecato, ora si ha sito arco intersecato,
    #segmento con inizio in intersezione, nuovo sito, segmento con inizio in intersezione, sito arco intersecato
    def add_arc(self, new_arc_site, crossed_arc_node):
        crossed_arc_site = crossed_arc_node.item
        #QUI SFRUTTA L'ORDINAMENTO IN Y DEGLI EVENTI: va aggiunto a new_arc_site.y un vettore di lung=new_arc_site.dist_to_par(crossed_arc_site) nel verso opposto a quello di sweep
        #se la sl va verso y crescenti la y del circle event sarà cc.y-r
        #se la sl va verso y decrescenti la y del circle event sarà cc.y+r
        cross_point = Point(new_arc_site.x, new_arc_site.y-new_arc_site.dist_to_par(crossed_arc_site))
        new_lhalfedge=Segment(pl=crossed_arc_site, pr=new_arc_site, start=cross_point)
        new_rhalfedge=Segment(pl=new_arc_site, pr=crossed_arc_site, start=cross_point)
        crossed_arc_node.item = new_lhalfedge
        crossed_arc_node.add_lchild(crossed_arc_site)
        crossed_arc_node.add_rchild(new_rhalfedge)
        crossed_arc_node.r.add_lchild(new_arc_site)
        crossed_arc_node.r.add_rchild(crossed_arc_site)
     
    def rm_arc(self, new_edge_start, arcs_nodes, edges_nodes):#in realtà potrebbe calcolarseli tutti, ma mi sembrava più efficiente passarglieli siccome già noti
        #salvataggi dei valori da ritornare prima dell'aggiornamento dell'albero
        (live_larc_node, dead_arc_node, live_rarc_node)=(arcs_nodes[0], arcs_nodes[1], arcs_nodes[2])
        #aggiornamento dell'albero
            #salvataggi dei riferimenti per nuovo assegnamento
        if dead_arc_node.is_rchild():
            (live_cut_branch, mid_edge_node, top_edge_node)=(edges_nodes[0].l, edges_nodes[0], edges_nodes[1])  
        else:
            (live_cut_branch, mid_edge_node, top_edge_node)=(edges_nodes[1].r, edges_nodes[1], edges_nodes[0])  
            #nuovo assegnamento
        top_edge_node.item = Segment(pl=live_larc_node.item, pr=live_rarc_node.item, start=new_edge_start, end=None)
        if mid_edge_node.is_rchild(): #aggiornamento puntatore padre->figlio
            mid_edge_node.parent.r=live_cut_branch #non usa metodo add_rchild perchè non aggiunge un nodo a partire da un item
        else:
            mid_edge_node.parent.l=live_cut_branch
        live_cut_branch.parent=mid_edge_node.parent #aggiornamento puntatore figlio->padre

In [11]:
p1=Point(-2,0)
p2=Point(-1.1,0)
p3=Point(0,0)
p4=Point(1,0)
p5=Point(2,0)
bl=Beachline()
print(bl.str_tree())
bl.add_start_arc(p3, bl.get_nearest_arc_node_by_x(p3))
print(bl.str_tree())
bl.add_start_arc(p5, bl.get_nearest_arc_node_by_x(p5))
print(bl.str_tree())
bl.add_start_arc(p4, bl.get_nearest_arc_node_by_x(p4))
print(bl.str_tree())
bl.add_start_arc(p1, bl.get_nearest_arc_node_by_x(p1))
print(bl.str_tree())
bl.add_start_arc(p2, bl.get_nearest_arc_node_by_x(p2))
print(bl.str_tree())

Empty beachline.

Path: root. Arc. (0.00,0.00). Circle event: None

Path: root. Edge. ((1.00,-inf) -> None, left focus (0.00,0.00), right focus (2.00,0.00))
Path: root->l. Arc. (0.00,0.00). Circle event: None
Path: root->r. Arc. (2.00,0.00). Circle event: None

Path: root. Edge. ((0.50,-inf) -> None, left focus (0.00,0.00), right focus (1.00,0.00))
Path: root->l. Arc. (0.00,0.00). Circle event: None
Path: root->r. Edge. ((1.50,-inf) -> None, left focus (1.00,0.00), right focus (2.00,0.00))
Path: root->r->l. Arc. (1.00,0.00). Circle event: None
Path: root->r->r. Arc. (2.00,0.00). Circle event: None

Path: root. Edge. ((0.50,-inf) -> None, left focus (0.00,0.00), right focus (1.00,0.00))
Path: root->l. Edge. ((-1.00,-inf) -> None, left focus (-2.00,0.00), right focus (0.00,0.00))
Path: root->l->l. Arc. (-2.00,0.00). Circle event: None
Path: root->l->r. Arc. (0.00,0.00). Circle event: None
Path: root->r. Edge. ((1.50,-inf) -> None, left focus (1.00,0.00), right focus (2.00,0.00))
Path: ro

In [12]:
first_arc_site = Point(5, 10)
bl = Beachline(first_arc_site)

In [13]:
print(bl.item.x, bl.item.y, bl.parent, bl.l, bl.r, bl.c_event)

5 10 None None None None


In [14]:
second_site = Point(first_arc_site.x+2, first_arc_site.y-2) #così facendo, le halfedge sono costruite con start=[first_arc_site.x+2, first_arc_site.y]
crossed_arc_node = bl.get_arc_node_on_site(second_site)
bl.add_arc(second_site, crossed_arc_node)
print("Root. Segment with start point: [", bl.item.start.x, ",", bl.item.start.y, "]") #1st-2nd site cross point
print("Root.r. Segment with start point: [", bl.r.item.start.x, ",", bl.r.item.start.y, "]") #1st-2nd site cross point
print("Root.l. Point: [", bl.l.item.x, ",", bl.l.item.y, "]") #1st site
print("Root.r.r. Point: [", bl.r.r.item.x, ",", bl.r.r.item.y, "]") #1st site
print("Root.r.l. Point: [", bl.r.l.item.x, ",", bl.r.l.item.y, "]") #2nd site

Root. Segment with start point: [ 7 , 6.0 ]
Root.r. Segment with start point: [ 7 , 6.0 ]
Root.l. Point: [ 5 , 10 ]
Root.r.r. Point: [ 5 , 10 ]
Root.r.l. Point: [ 7 , 8 ]


In [15]:
third_site = Point(6, 6)
crossed_arc_node = bl.get_arc_node_on_site(third_site)
bl.add_arc(third_site, crossed_arc_node)
print(bl.str_tree())

Path: root. Edge. ((7.00,6.00) -> None, left focus (5.00,10.00), right focus (7.00,8.00))
Path: root->l. Edge. ((6.00,3.88) -> None, left focus (5.00,10.00), right focus (6.00,6.00))
Path: root->l->l. Arc. (5.00,10.00). Circle event: None
Path: root->l->r. Edge. ((6.00,3.88) -> None, left focus (6.00,6.00), right focus (5.00,10.00))
Path: root->l->r->l. Arc. (6.00,6.00). Circle event: None
Path: root->l->r->r. Arc. (5.00,10.00). Circle event: None
Path: root->r. Edge. ((7.00,6.00) -> None, left focus (7.00,8.00), right focus (5.00,10.00))
Path: root->r->l. Arc. (7.00,8.00). Circle event: None
Path: root->r->r. Arc. (5.00,10.00). Circle event: None



In [16]:
first_leaf = bl.l.l
print(first_leaf.item.x, first_leaf.item.y)
second_leaf = bl.get_rleaf_and_rparent(first_leaf)[0]
print(second_leaf.item.x, second_leaf.item.y) #2nd site
third_leaf = bl.get_rleaf_and_rparent(second_leaf)[0]
print(third_leaf.item.x, third_leaf.item.y) #3rd site
fourth_leaf = bl.get_rleaf_and_rparent(third_leaf)[0]
print(fourth_leaf.item.x, fourth_leaf.item.y) #2nd site
fifth_leaf = bl.get_rleaf_and_rparent(fourth_leaf)[0]
print(fifth_leaf.item.x, fifth_leaf.item.y) #1st site
sixth_leaf = bl.get_rleaf_and_rparent(fifth_leaf)[0]
print(sixth_leaf) #None
print(fourth_leaf==fifth_leaf.get_lleaf_and_lparent()[0])
print(third_leaf==fourth_leaf.get_lleaf_and_lparent()[0])
print(second_leaf==third_leaf.get_lleaf_and_lparent()[0])
print(first_leaf==second_leaf.get_lleaf_and_lparent()[0])
print(first_leaf.get_lleaf_and_lparent()[0]) #None

5 10
6 6
5 10
7 8
5 10
None
True
True
True
True
None


In [17]:
new_edge_start=Point(2,9)
(lleaf, ledge_node)=second_leaf.get_lleaf_and_lparent()
(rleaf, redge_node)=second_leaf.get_rleaf_and_rparent()
bl.rm_arc(new_edge_start, [lleaf,second_leaf,rleaf], [ledge_node, redge_node]) #second leaf è un figlio sinistro
print(bl.str_tree())

Path: root. Edge. ((7.00,6.00) -> None, left focus (5.00,10.00), right focus (7.00,8.00))
Path: root->l. Edge. ((2.00,9.00) -> None, left focus (5.00,10.00), right focus (5.00,10.00))
Path: root->l->l. Arc. (5.00,10.00). Circle event: None
Path: root->l->r. Arc. (5.00,10.00). Circle event: None
Path: root->r. Edge. ((7.00,6.00) -> None, left focus (7.00,8.00), right focus (5.00,10.00))
Path: root->r->l. Arc. (7.00,8.00). Circle event: None
Path: root->r->r. Arc. (5.00,10.00). Circle event: None



In [18]:
nnew_edge_start=Point(3,4)
(lleaf, ledge_node)=third_leaf.get_lleaf_and_lparent()
(rleaf, redge_node)=third_leaf.get_rleaf_and_rparent()
bl.rm_arc(new_edge_start, [lleaf,third_leaf,rleaf], [ledge_node, redge_node]) #fifth leaf è un figlio destro
print(bl.str_tree())

Path: root. Edge. ((2.00,9.00) -> None, left focus (5.00,10.00), right focus (7.00,8.00))
Path: root->l. Arc. (5.00,10.00). Circle event: None
Path: root->r. Edge. ((7.00,6.00) -> None, left focus (7.00,8.00), right focus (5.00,10.00))
Path: root->r->l. Arc. (7.00,8.00). Circle event: None
Path: root->r->r. Arc. (5.00,10.00). Circle event: None



In [19]:
class Event(object):
    
    def __init__(self, arc_site_or_nodes): #sites: tupla dei siti associati agli archi coinvolti nell'evento (ordinati)
        if isinstance(arc_site_or_nodes, Point): 
            self.coord=arc_site_or_nodes
            self.arcs_nodes=None
        else:
            self.arcs_nodes=[arc_site_or_nodes[0], arc_site_or_nodes[2], arc_site_or_nodes[4]]
            self.edges_nodes=[arc_site_or_nodes[1], arc_site_or_nodes[3]]
            self.vertex_coord=Circle.center(self.arcs_nodes[0].item, self.arcs_nodes[1].item, self.arcs_nodes[2].item)
            #QUI SFRUTTA L'ORDINAMENTO IN Y DEGLI EVENTI: se sweep verso y crescenti uso circumtop, viceversa circumbottom
            self.coord=Point(self.vertex_coord.x,self.vertex_coord.y+self.arcs_nodes[0].item.dist_to_point(self.vertex_coord))
            
    def __str__(self):
        if(self.arcs_nodes==None): #site event
            return "Site event. Site: "+str(self.coord)  
        else: #circle event
            return "Circle event. Coord:"+str(self.coord)+". Arc site:"+str(self.arcs_nodes[1].item)
    
    def __eq__(self, e):
        return isinstance(e, Event) and self.arcs_nodes==e.arcs_nodes and self.coord==e.coord
    
    #QUI DEFINISCE L'ORDINAMENTO IN Y DEGLI EVENTI: in base a come ordino gli eventi, la sl andrà verso y crescenti o decrescenti
    #inserisce i nuovi circle come primi eventi a tale y. Penso sia giusto gestirli prima dei site event, in modo da avere la beach aggiornata
    def add(self, es):
        i=Ordering.go_to_i(es, self, Ordering.is_coord_y_greater)
        es.insert(i, self)
    
    def rm(self, es):
        i = Ordering.go_to_i(es, self, Ordering.is_coord_y_greater)
        while True:
            #se non esiste un altro evento alla stessa y elimina subito l'evento corrente, se no effettua il controllo
            if i==len(es)-1 or self.coord.y != es[i+1].coord.y or self==es[i]: 
                return es.pop(i)
            i+=1            

In [20]:
class Voronoi(object):
    
    def __init__(self, sites):
        self.sites=sites  #siti di V, obj Point
        self.events=[]
        self.iterations=0 #inserito per il calcolo delle iterazioni, ma non necessario alla creazione del diagramma
        for site in sites:
            Event(site).add(self.events)
        self.beach=Beachline()
        self.edges=[]
             
    
    #rimuove dalla lista degli eventi l'evento cicle il cui arco associato all'indice della beachline argomento è 
    #quello centrale
    def rm_circle_event(self, arc_node):
        if arc_node.c_event != None:
            arc_node.c_event.rm(self.events)
            arc_node.c_event = None
    
    #NB: un circle event è originato da tre siti di V adiacenti. In esso viene eliminato l'arco centrale. Da notare
    #che l'arco centrale non è associabile a priori in modo banale ad uno dei tre punti
    #Il metodo controlla gli archi interessati da un cambiamento della beachline e verifica se ci sono dei circle
    #event loro associati. Se ci sono, aggiunge l'evento alla coda salvando i siti degli archi nell'ordine di comparsa
    #e salvando un riferimento all'evento 
    def add_circle_event(self, arc_node, sweep_y):
        larc_node, ledge_node = arc_node.get_lleaf_and_lparent()
        rarc_node, redge_node = arc_node.get_rleaf_and_rparent()
        if larc_node==None or rarc_node==None:
            return #"No event: first or last arc"
        if larc_node.item==rarc_node.item:
            return #"No event: lefter arc site equals righter arc site"
        cc = Circle.center(larc_node.item, arc_node.item, rarc_node.item)
        if cc.y==np.inf:
            return #"No event: the three arcs sites are on the same line"
        #QUI SFRUTTA L'ORDINAMENTO IN Y DEGLI EVENTI: va preso l'estremo della circonferenza nella direzione di sweep
        #se la sl va verso y crescenti la y del circle event sarà cc.y+r
        #se la sl va verso y decrescenti la y del circle event sarà cc.y-r
        event_point=Point(cc.x, cc.y+arc_node.item.dist_to_point(cc))
        #QUI SFRUTTA L'ORDINAMENTO IN Y DEGLI EVENTI: un evento è sotto la sl se deve essere ancora incontrato durante lo sweep, e viceversa
        #se la sl va verso y crescenti un evento è sotto la sl se ha y minore di quella della sl
        #se la sl va verso y decrescenti un evento è sotto la sl se ha y maggiore di quella della sl
        if event_point.y<sweep_y-1e-10:
            return #"No event: circle event y", event_point.y ,"is below sweepline y",sweep_y
        if ledge_node.item.get_intersection(redge_node.item)==None:
            return #"Edges do not cross"
        arc_node.c_event = Event([larc_node, ledge_node, arc_node, redge_node, rarc_node])
        arc_node.c_event.add(self.events)
        #return arc_node.c_event
    
    def manage_start_site_event(self):
        event=self.events.pop(0)
        self.beach.add_start_arc(event.coord, self.beach.get_nearest_arc_node_by_x(event.coord))
    
    def manage_site_event(self):
        event=self.events.pop(0) #tolgo subito l'evento gestito; facendolo dopo potrei rimuovere un circle event aggiunto
        crossed_arc_node=self.beach.get_arc_node_on_site(event.coord)
        rarc_node=crossed_arc_node.get_rleaf_and_rparent()[0]
        larc_node=crossed_arc_node.get_lleaf_and_lparent()[0]
        #rimozione evento dal nodo che diventerà radice del ramo aggiunto all'albero binario
        self.rm_circle_event(crossed_arc_node)
        #eliminazione eventi associati a foglia destra e sinistra nella vecchia configurazione. Necessaria perchè gli eventi in
        #esse immagazzinati hanno un riferimento al nodo dell'arco intersecato, che ora diventerà il nodo segmento radice
        if larc_node!=None:
            self.rm_circle_event(larc_node)
        if rarc_node!=None:
            self.rm_circle_event(rarc_node)
        self.beach.add_arc(event.coord, crossed_arc_node)
        #aggiornamento degli eventi associati a foglia destra e sinistra nella vecchia configurazione. In tale modo sono assegnati
        #all'oggetto evento i riferimenti ai nodi arco aggiornati
        if larc_node!=None:
            self.add_circle_event(larc_node, event.coord.y)
        if rarc_node!=None:
            self.add_circle_event(rarc_node, event.coord.y)
        #controllo esistenza ed eventuale aggiunta eventi di cerchio dei due archi in cui è diviso l'arco intersecato
        self.add_circle_event(crossed_arc_node.l, event.coord.y)
        self.add_circle_event(crossed_arc_node.r.r, event.coord.y)
        
    def manage_circle_event(self):
        event=self.events.pop(0) #tolgo subito l'evento gestito; facendolo dopo potrei rimuovere un circle event aggiunto
        (dead_edgeA, dead_edgeB)=(event.edges_nodes[0].item, event.edges_nodes[1].item)
        (dead_edgeA.end, dead_edgeB.end)=(event.vertex_coord, event.vertex_coord) 
        self.beach.rm_arc(event.vertex_coord, event.arcs_nodes, event.edges_nodes)
        self.edges.append(dead_edgeA)
        self.edges.append(dead_edgeB)
        self.rm_circle_event(event.arcs_nodes[0])
        self.rm_circle_event(event.arcs_nodes[2])
        self.add_circle_event(event.arcs_nodes[0], event.coord.y)
        self.add_circle_event(event.arcs_nodes[2], event.coord.y)

    #se eseguito al temine di run() estrae le semirette presenti nell'albero ad algorimo terminato. Le semirette (segmenti non t
    #terminati) sono quelli presenti a inizio lista edges con start.y=-np.inf (associate ai siti con stessa y gestiti nei primi step)
    #e quelli presenti nell'albero ad algoritmo terminato 
    def extract_edges(self, root=None):
        root=self.beach if root==None else root
        if not root.is_leaf():
            self.edges.append(root.item)
            self.extract_edges(root.l)
            self.extract_edges(root.r)
            
    def run(self):
        first_y=self.events[0].coord.y
        self.iterations+=1
        while self.events!=[] and self.events[0].coord.y==first_y:
            self.manage_start_site_event()
            self.iterations+=1
        while self.events!=[]:
            self.manage_site_event() if self.events[0].arcs_nodes==None else self.manage_circle_event()
            self.iterations+=1
        #self.extract_edges()
            
    def plot_edges(self, x_range=(-1, 1), y_range=(-1,1), file_name="voronoi_edges.png"):
        self.run()
        lines = [[(edge.start.x, edge.start.y),(edge.end.x, edge.end.y)] for edge in self.edges]
        lc = mc.LineCollection(lines)
        fig, ax = plt.subplots()
        ax.axis([*x_range, *y_range])
        ax.add_collection(lc)
        ax.margins(0.1)
        xs, ys = zip(*[[p.x,p.y] for p in self.sites])
        ax.plot(xs, ys, 'ro')
        fig.savefig(file_name)
    
    #non gestisce il caso di due singoli siti, non riuscendo ad associare essi un poligono (necessari almeno due segmenti associati
    #a ciascun sito)
    def plot_patches(self, x_range=(-1, 1), y_range=(-1,1), file_name="voronoi_patches.png"): 
        self.run()
        pts = self.sites
        pts_dict = defaultdict(list)
        patches = []
        colors = []
        for edge in self.edges:
            pts_dict[edge.pl].append((edge.start, edge.end))
            pts_dict[edge.pr].append((edge.start, edge.end))
        for center, v_raw in pts_dict.items():
            starts, ends = zip(*v_raw)
            vertices = set(starts + ends)
            vertices = sorted(vertices, key=lambda p: np.arctan2(p.y-center.y,p.x-center.x))
            vertices = [(v.x, v.y) for v in vertices]
            patches.append(Polygon(vertices, True))
            colors.append(center.dist_to_point(Point(0,0)))
        fig, ax = plt.subplots()
        colors = 100*np.random.rand(len(patches))
        pc = PatchCollection(patches, cmap=jet, alpha=0.2)
        pc.set_array(np.array(colors))
        ax.axis([*x_range, *y_range])
        ax.add_collection(pc)
        ax.margins(0.1)
        xs, ys = zip(*[(p.x, p.y) for p in pts])
        ax.plot(xs, ys, 'ro', markersize=1)
        fig.savefig(file_name)

In [21]:
#Test Voronoi: coda degli eventi
v=Voronoi((first_leaf.item, second_leaf.item, third_leaf.item))
print("sites:", *[str([site.x, site.y]) for site in v.sites])
print("1st event site:", v.events[0].coord.x, v.events[0].coord.y) #3rd site
print("2nd event site:", v.events[1].coord.x, v.events[1].coord.y) #2nd site
print("3rd event site:", v.events[2].coord.x, v.events[2].coord.y) #1st site
#print("Beachline after init (empty):", v.beach.item) #None

sites: [5, 10] [6, 6] [5, 10]
1st event site: 6 6
2nd event site: 5 10
3rd event site: 5 10


In [22]:
v.events[1].rm(v.events) #rimozione in mezzo alla coda
print("1st event site:", v.events[0].coord.x, v.events[0].coord.y) #3rd site
print("2nd event site:", v.events[1].coord.x, v.events[1].coord.y, "\n") #1st site
Event(Point(7, 7)).add(v.events) #aggiunta in mezzo alla coda
print("1st event site:", v.events[0].coord.x, v.events[0].coord.y) #3rd site
print("2nd event site:", v.events[1].coord.x, v.events[1].coord.y) #new_ev_1 site
print("3rd event site:", v.events[2].coord.x, v.events[2].coord.y) #1st site

1st event site: 6 6
2nd event site: 5 10 

1st event site: 6 6
2nd event site: 7 7
3rd event site: 5 10


In [23]:
v.events[0].rm(v.events) #rimozione a inizio coda
print("1st event site:", v.events[0].coord.x, v.events[0].coord.y) #new_ev_1 site
print("2nd event site:", v.events[1].coord.x, v.events[1].coord.y, "\n") #1st site
Event(Point(0, -1)).add(v.events) #aggiunta a inizio coda
print("1st event site:", v.events[0].coord.x, v.events[0].coord.y) #new_ev_2 site
print("2nd event site:", v.events[1].coord.x, v.events[1].coord.y) #new_ev_1 site
print("3rd event site:", v.events[2].coord.x, v.events[2].coord.y) #1st site

1st event site: 7 7
2nd event site: 5 10 

1st event site: 0 -1
2nd event site: 7 7
3rd event site: 5 10


In [24]:
v.events[2].rm(v.events) #rimozione a fondo coda
print("1st event site:", v.events[0].coord.x, v.events[0].coord.y) #new_ev_2_site
print("2nd event site:", v.events[1].coord.x, v.events[1].coord.y, "\n") #new_ev_1 site
Event(Point(-6, 9.3)).add(v.events) #aggiunta a fondo coda
print("1st event site:", v.events[0].coord.x, v.events[0].coord.y) #new_ev_2 site
print("2nd event site:", v.events[1].coord.x, v.events[1].coord.y) #new_ev_1 site
print("3rd event site:", v.events[2].coord.x, v.events[2].coord.y) #new_ev_3 site

1st event site: 0 -1
2nd event site: 7 7 

1st event site: 0 -1
2nd event site: 7 7
3rd event site: -6 9.3


In [25]:
Event(Point(-5, 9.3)).add(v.events) #aggiunta alla stessa y 
print("1st event site:", v.events[0].coord.x, v.events[0].coord.y) #new_ev_2 site
print("2nd event site:", v.events[1].coord.x, v.events[1].coord.y) #new_ev_1 site
print("3rd event site:", v.events[2].coord.x, v.events[2].coord.y) #new_ev_4 site
print("4th event site:", v.events[3].coord.x, v.events[2].coord.y) #new_ev_3 site

1st event site: 0 -1
2nd event site: 7 7
3rd event site: -5 9.3
4th event site: -6 9.3


In [26]:
v.events[3].rm(v.events) #rimozione alla stessa y
print("1st event site:", v.events[0].coord.x, v.events[0].coord.y) #new_ev_2 site
print("2nd event site:", v.events[1].coord.x, v.events[1].coord.y) #new_ev_1 site
print("3rd event site:", v.events[2].coord.x, v.events[2].coord.y) #new_ev_4 site

1st event site: 0 -1
2nd event site: 7 7
3rd event site: -5 9.3


In [27]:
### controllo di add_start_arc e add_arc

v1=Voronoi([Point(0,2), Point(np.sqrt(2),np.sqrt(2)), Point(2,0), Point(3,0)])
first_site=v1.events[0].coord
first_site_arc_node=v1.beach.get_nearest_arc_node_by_x(first_site)
v1.beach.add_start_arc(first_site, first_site_arc_node)
print(v1.beach.str_tree())

second_site=v1.events[1].coord
nearest_site_arc_node=v1.beach.get_nearest_arc_node_by_x(second_site)
v1.beach.add_start_arc(second_site, nearest_site_arc_node)
print(v1.beach.str_tree())

third_site=v1.events[2].coord
crossed_arc_node=v1.beach.get_arc_node_on_site(third_site)
v1.beach.add_arc(third_site, crossed_arc_node)
print(v1.beach.str_tree())

fourth_site=v1.events[3].coord
crossed_arc_node=v1.beach.get_arc_node_on_site(fourth_site)
v1.beach.add_arc(fourth_site, crossed_arc_node)
print(v1.beach.str_tree())

Path: root. Arc. (3.00,0.00). Circle event: None

Path: root. Edge. ((2.50,-inf) -> None, left focus (2.00,0.00), right focus (3.00,0.00))
Path: root->l. Arc. (2.00,0.00). Circle event: None
Path: root->r. Arc. (3.00,0.00). Circle event: None

Path: root. Edge. ((2.50,-inf) -> None, left focus (2.00,0.00), right focus (3.00,0.00))
Path: root->l. Edge. ((1.41,0.59) -> None, left focus (2.00,0.00), right focus (1.41,1.41))
Path: root->l->l. Arc. (2.00,0.00). Circle event: None
Path: root->l->r. Edge. ((1.41,0.59) -> None, left focus (1.41,1.41), right focus (2.00,0.00))
Path: root->l->r->l. Arc. (1.41,1.41). Circle event: None
Path: root->l->r->r. Arc. (2.00,0.00). Circle event: None
Path: root->r. Arc. (3.00,0.00). Circle event: None

Path: root. Edge. ((2.50,-inf) -> None, left focus (2.00,0.00), right focus (3.00,0.00))
Path: root->l. Edge. ((1.41,0.59) -> None, left focus (2.00,0.00), right focus (1.41,1.41))
Path: root->l->l. Edge. ((0.00,-0.00) -> None, left focus (2.00,0.00), righ

In [28]:
arc_nodes=v1.beach.get_leaves()
for arc_node in arc_nodes:
    print(arc_node.str_node([]))

Arc. (2.00,0.00). Circle event: None
Arc. (0.00,2.00). Circle event: None
Arc. (2.00,0.00). Circle event: None
Arc. (1.41,1.41). Circle event: None
Arc. (2.00,0.00). Circle event: None
Arc. (3.00,0.00). Circle event: None


In [29]:
print("Before checking for events")
print("\n# Arc nodes")
for arc_node in v1.beach.get_leaves():
    print(arc_node.str_node([]))
print("\n# Event queue")
print([str(event) for event in v1.events])

Before checking for events

# Arc nodes
Arc. (2.00,0.00). Circle event: None
Arc. (0.00,2.00). Circle event: None
Arc. (2.00,0.00). Circle event: None
Arc. (1.41,1.41). Circle event: None
Arc. (2.00,0.00). Circle event: None
Arc. (3.00,0.00). Circle event: None

# Event queue
['Site event. Site: (3.00,0.00)', 'Site event. Site: (2.00,0.00)', 'Site event. Site: (1.41,1.41)', 'Site event. Site: (0.00,2.00)']


In [30]:
#non elimina gli archi intermedi anche una volta morti siccome non gestisco i circle event
#nB: coordinate dei circle event non sono il centro della circonferenza ma il suo estremo
print("\nWhile checking for circle events")
sweep_y=1 #da tunare per verificare se l'evento viene scartato quando cade nel range già scorso
for arc_node in v1.beach.get_leaves():
    print("*", v1.add_circle_event(arc_node, sweep_y))
    print(arc_node.str_node())
print("\n", [str(event) for event in v1.events])


While checking for circle events
* None
Arc. (2.00,0.00). Circle event: None
* None
Arc. (0.00,2.00). Circle event: None
* None
Arc. (2.00,0.00). Event: Circle event. Coord:(0.00,2.00). Arc site:(2.00,0.00)
* None
Arc. (1.41,1.41). Circle event: None
* None
Arc. (2.00,0.00). Event: Circle event. Coord:(2.50,2.19). Arc site:(2.00,0.00)
* None
Arc. (3.00,0.00). Circle event: None

 ['Site event. Site: (3.00,0.00)', 'Site event. Site: (2.00,0.00)', 'Site event. Site: (1.41,1.41)', 'Circle event. Coord:(0.00,2.00). Arc site:(2.00,0.00)', 'Site event. Site: (0.00,2.00)', 'Circle event. Coord:(2.50,2.19). Arc site:(2.00,0.00)']


In [31]:
### controllo di rm_circle_event
print("\nWhile checking for circle events")
print("\nEvent queue before removal: ")
print([str(event) for event in v1.events])
for arc_node in v1.beach.get_leaves():
    print("\nCircle event before removal: "+arc_node.str_node([]))
    v1.rm_circle_event(arc_node)
    print("Circle event after removal: "+arc_node.str_node([]))
print("\nEvent queue after removal: ")
print([str(event) for event in v1.events])


While checking for circle events

Event queue before removal: 
['Site event. Site: (3.00,0.00)', 'Site event. Site: (2.00,0.00)', 'Site event. Site: (1.41,1.41)', 'Circle event. Coord:(0.00,2.00). Arc site:(2.00,0.00)', 'Site event. Site: (0.00,2.00)', 'Circle event. Coord:(2.50,2.19). Arc site:(2.00,0.00)']

Circle event before removal: Arc. (2.00,0.00). Circle event: None
Circle event after removal: Arc. (2.00,0.00). Circle event: None

Circle event before removal: Arc. (0.00,2.00). Circle event: None
Circle event after removal: Arc. (0.00,2.00). Circle event: None

Circle event before removal: Arc. (2.00,0.00). Event: Circle event. Coord:(0.00,2.00). Arc site:(2.00,0.00)
Circle event after removal: Arc. (2.00,0.00). Circle event: None

Circle event before removal: Arc. (1.41,1.41). Circle event: None
Circle event after removal: Arc. (1.41,1.41). Circle event: None

Circle event before removal: Arc. (2.00,0.00). Event: Circle event. Coord:(2.50,2.19). Arc site:(2.00,0.00)
Circle eve

In [32]:
v_two_points=[Point(0,-1), Point(0,1)]
v_four_points=[Point(0,0), Point(0,1), Point(0,2), Point(0,3)]
v_triangle=[Point(0,0), Point(1,1), Point(-1,1)]
v_square=[Point(0,-1), Point(1,0), Point(-1,0), Point(0,1)]
v_centered_square=[Point(0,0), Point(0,-1), Point(1,0), Point(-1,0), Point(0,1)]
v_cross=[Point(0,-1), Point(1,0), Point(-1,0), Point(0,2)]
v_octagon=[Point(0,-2), Point(-np.sqrt(2),-np.sqrt(2)), Point(np.sqrt(2),-np.sqrt(2)), Point(-2,0), Point(2,0),Point(-np.sqrt(2),np.sqrt(2)), Point(np.sqrt(2),np.sqrt(2)), Point(0,2)]
v_centered_octagon=[Point(0,0), Point(0,-2), Point(-np.sqrt(2),-np.sqrt(2)), Point(np.sqrt(2),-np.sqrt(2)), Point(-2,0), Point(2,0),Point(-np.sqrt(2),np.sqrt(2)), Point(np.sqrt(2),np.sqrt(2)), Point(0,2)]

In [33]:
h_two_points=[Point(0,0), Point(1,0)]
h_four_points=[Point(0,0), Point(1,0), Point(2,0), Point(3,0)]
h_triangle=[Point(-1,-1), Point(1,-1), Point(0,1)]
h_square=[Point(-1,-1), Point(1,-1), Point(-1,1), Point(1,1)]
h_centered_square=[Point(-1,-1), Point(1,-1), Point(-1,1), Point(1,1), Point(0,0)]
x1=np.cos(np.pi/8); y1=np.sin(np.pi/8);x2=np.cos(3*np.pi/8);y2=np.sin(3*np.pi/8)
h_octagon=[Point(x1,y1), Point(x2,y2), Point(-x1,y1), Point(-x2,y2), Point(-x1,-y1), Point(-x2,-y2), Point(x1,-y1), Point(x2,-y2)]
h_centered_octagon=[Point(0,0), Point(x1,y1), Point(x2,y2), Point(-x1,y1), Point(-x2,y2), Point(-x1,-y1), Point(-x2,-y2), Point(x1,-y1), Point(x2,-y2)]

In [34]:
(xmin, xmax, ymin, ymax) = (-2, 2, -2, 2)
n_points=10
xs = np.random.uniform(xmin, xmax, n_points)
ys = np.random.uniform(ymin, ymax, n_points)
random = [Point(x, y) for x, y in zip(xs, ys)]

In [35]:
#v2=Voronoi(v_two_points)
#v2=Voronoi(v_four_points)
#v2=Voronoi(v_triangle)
#v2=Voronoi(v_square)
#v2=Voronoi(v_centered_square)
#v2=Voronoi(v_cross)
#v2=Voronoi(v_octagon)
#v2=Voronoi(v_centered_octagon)

In [36]:
#v2=Voronoi(h_two_points)
#v2=Voronoi(h_four_points)
#v2=Voronoi(h_triangle)
#v2=Voronoi(h_square)
#v2=Voronoi(h_centered_square)
#v2=Voronoi(h_octagon)
#v2=Voronoi(h_centered_octagon)

In [37]:
def main(sites, x_range=(-1,1), y_range=(-1,1)):
    print("started")
    time0 = time.time()
    xm=(x_range[1]-x_range[0])/2
    ym=(y_range[1]-y_range[0])/2
    diag=np.sqrt(np.power(x_range[1]-x_range[0], 2) + np.power(y_range[1]-y_range[0], 2))
    closing_points=[Point(xm, y_range[0]-diag), Point(x_range[1]+diag, ym), Point(xm, y_range[1]+diag), Point(x_range[0]-diag, ym)]
    [print(str(point)) for point in sites+closing_points]
    v=Voronoi(sites+closing_points)
    print(v.beach.str_tree())
    #v.plot_patches(x_range, y_range)
    v.plot_edges(x_range, y_range, file_name="voronoi_segments.png")
    time1 = time.time()
    print("ended")
    print("it took %.4f seconds and %s iterations" % (time1 - time0, v.iterations))

In [38]:
x_range=[-5, 5]
y_range=[-5, 5]
points=v_octagon
main(points, x_range, y_range)

started
(0.00,-2.00)
(-1.41,-1.41)
(1.41,-1.41)
(-2.00,0.00)
(2.00,0.00)
(-1.41,1.41)
(1.41,1.41)
(0.00,2.00)
(5.00,-19.14)
(19.14,5.00)
(5.00,19.14)
(-19.14,5.00)
Empty beachline.

ended
it took 0.1937 seconds and 27 iterations


## GESTITI:

### site event con nuovo sito con sulla verticale l'intersezione di due archi adiacenti 
Vado a sezionare coon il nuovo arco pc uno dei due archi soprastanti pl, pr (destro o sinistro, in base al confronto implementato. Non è importante quale dei due. Supponendo di intersecare l'arco destro la beachline va da pl, pr --> pl, pr', pc, pr''). Contestualmente si crea un circle event per l'arco fittizio creato . Come primo evento successivo gerstirò tale circle event aggiornando la beach (Nell'esempio eliminerò pr', andando da pl, pr', pc, pr''-> pl, pc, pr'').

### eventi alla stessa y (sia circle che site)
Penso sia meglio gestire prima i circle che i site, in modo da avere la beach aggiornata quando gestisco i site. 
Non penso più site event alla stessa y diano problemi, qualsiasi sia l'ordine con il quale sono gestiti
Non penso più circle event alla stessa y diano problemi, anche se questo non è ovvio. In particolare andrebbe investigato il caso in cui, eliminando un arco, aggiungo un circle event (che verrà eseguito come circle event successivo, coinvolgente un arco che deve essere eliminato da un circle event alla stessa y)

### eventi alla stessa [x,y] sia circle che site
Site event: è stupido. L'utente non dovrebbe inserirne. Nel caso si possono eliminare siti uguali dall'input fornito
Cirle event: non dovrebbero dare problemi neanche questi, anche se meno ovvio.

### più siti iniziali alla stessa y
* Possibile cereare il relativo albero separatamente. Le foglie sono inserite controllando quale è il sito più vicino in x (e non quale è l'arco soprastante, non esistendo ancora un arco soprastante. Se equisistante da due siti adiacenti effettuo l'assegnazione a uno dei due. Non è importante a quale dei due). L'arco che cade più vicino in x al nuovo sito è sostituito con un nodo segmento a due figli (nuovo sito figlio destro e sito più vicino figlio sinistro se il nuovo sito si trova a destra del sito più vicino, e viceversa). Il nodo padre è il segmento tra nuovo sito e sito adiacente (ci associo punto all'infinito o cosa?). Dovrò anche aggiornare il nodo a monte assegnato al segmento tra il sito più vicino e il sito che precedentemente gli stava a dx (se nuovo sito a destra del sito più vicino. Ora il sito di destra è il nuovo sito) o a sx (se nuovo sito a sinistra del sito più vicino. Ora il sito di sinistra è il nuovo sito).

### chiusura algoritmo
* Metodo 1: estraggo i nodi segmenti rimasti nell'albero a termine algoritmo. Tronco i nodi che vanno all'infinito in modo che rimangano nella box e associo i vari segmenti in cui è diviso il bordo della box ai diversi siti. In teoria i segmenti che vanno all'infinito sono solo quelli all'inizio della lista delle edges (aggiunti al primo step dell'algoritmo per le coppie di siti alla stessa y) e quelli rimasti nell'albero al termine dell'algoritmo (segmenti semi infiniti)
* Metodo 2: aggiungo ai siti del diagramma di Voronoi quattro punti distanti a sufficienza dai quattro bordi della box di visualizzazione da non influenzare il diagramma visualizzato (dovrebbe essere sufficiente (xm, ymax+diag),(xm, ymin-diag),(xmin-diag, ym),(xmax+diag, ym) dove:  
    - xmin, xmax, ymin, ymax sono le coordinate dei lati della box, 
    - xm=(xax-xmin)/2, ym=(ymax-ymin)/2 sono le coordinate dei punti medi dei lati della box
    - diag è la diagonale della box
    Questo metodo automaticamente risolve anche i problemi legati a 
    - più punti iniziali alla stessa y (non ci sono se i punti sono disposti a rombo)
    - estrazione e troncamento dei segmenti non terminati (gli unici presenti sono fuori dal range di visualizzazione in quanto legati ai quattro assi di simmetria tra i punti aggiunti) 

## CAMBIAMENTI

### ottimizzazioni
* gestione puntatori in add_arc
* gestione condizione intersezione in get_intersection
* assegnamento hash univoco ai punti
* assegnamento colori diversi a celle adiacenti